### **Importing Necessary Libraries**

In [17]:
import tkinter as tk # GUI
from tkinter import messagebox # message box
import random # generating random numbers

### **RSA Functions**

In [18]:
# Calculates GCD of two numbers using Euclidean algorithm
def gcd(a, b):
    while b != 0:
        a, b = b, a % b
        # when the remainder of a / b is 0, b is the GCD
    return a

In [ ]:
# Calculates multiplicative inverse of two numbers
# finds modular inverse of e mod phi
# used to find the decryption key d
def multiplicative_inverse(e, phi):
    d = 0 # the modular inverse
    x1 = 0 # temporary variable
    x2 = 1 # temporary variable
    y1 = 1 # temporary variable
    temp_phi = phi # temporary variable

    while e > 0:
        # temp1 is the quotient of phi and e
        temp1 = temp_phi // e
        
        # temp2 is the remainder of phi and e
        temp2 = temp_phi - temp1 * e
        
        # temp_phi is now e
        temp_phi = e
        
        # e is now temp2
        e = temp2

        # updates the coefficient for e
        x = x2 - temp1 * x1
        
        # updates the coefficient for phi
        y = d - temp1 * y1

        # updates the temporary variables
        x2 = x1
        x1 = x
        d = y1
        y1 = y

    # if phi is 1 --> e and phi are coprime, and the modular inverse exists
    if temp_phi == 1:
        return d + phi

In [20]:
# Checks if a number is prime
def is_prime(num):
    # 0 and 1 are not prime
    if num < 2:
        return False
    
    # checks if num is divisible by any number from 2 to the square root of num (optimized)
    for n in range(2, int(num ** 0.5) + 2): 
        if num % n == 0:
            return False
    return True

In [21]:
# Generates public and private keys
def generate_key_pair(p, q):
    # checks if p and q are prime
    if not (is_prime(p) and is_prime(q)):
        raise ValueError('Both numbers must be prime.')
    
    # checks if p and q are not equal
    elif p == q:
        raise ValueError('p and q cannot be equal.')
    
    # n is the modulus for the public and private keys
    n = p * q
    
    # phi is the totient of n
    phi = (p - 1) * (q - 1)
    
    # chooses a random number e such that 1 < e < phi and e is coprime with phi
    e = random.randrange(1, phi)
    
    # checks if e and phi are coprime
    g = gcd(e, phi)
    
    # if e and phi are not coprime, choose another e. if e and phi are coprime, their GCD is 1
    while g != 1:
        e = random.randrange(1, phi)
        g = gcd(e, phi)
    
    # calculates the modular inverse of e mod phi
    d = multiplicative_inverse(e, phi)
    
    # returns the public and private keys
    return ((e, n), (d, n))

In [22]:
# Encrypts plaintext using public key
def encrypt(public_key, plaintext):
    # unpacks the public key
    key, n = public_key
    
    # converts each character in the plaintext to its ASCII value and raises it to the power of the key,
    # then takes the modulus of n
    cipher = [(ord(char) ** key) % n for char in plaintext]
    return cipher

In [23]:
# Decrypts ciphertext using private key
def decrypt(private_key, ciphertext):
    # unpacks the private key
    key, n = private_key
    
    # converts each character in the ciphertext to its ASCII value and raises it to the power of the key,
    # then takes the modulus of n
    plain = [chr((char ** key) % n) for char in ciphertext]
    return ''.join(plain)

### **GUI Functions**

In [24]:
# Generates keys and displays them
def generate_keys():
    try:
        p = int(entry_prime1.get())
        q = int(entry_prime2.get())
        public, private = generate_key_pair(p, q)
        public_key_var.set(f"Public Key: {public}")
        private_key_var.set(f"Private Key: {private}")
        global public_key, private_key
        public_key, private_key = public, private
    except ValueError as ve:
        messagebox.showerror("Error", str(ve))
    except Exception as e:
        messagebox.showerror("Error", "Invalid input or key generation failed.")

In [25]:
# Encrypts plaintext and displays ciphertext
def encrypt_message():
    plaintext = entry_plaintext.get()
    if not public_key:
        messagebox.showerror("Error", "Please generate keys first.")
        return
    encrypted_msg = encrypt(public_key, plaintext)
    encrypted_msg_str = ''.join(map(lambda x: str(x).zfill(4), encrypted_msg))  # Ensure each number is 4 digits
    entry_ciphertext.delete(0, tk.END)
    entry_ciphertext.insert(0, encrypted_msg_str)
    encryption_status.config(text="Encryption Successful.")
    decryption_status.config(text="") # Clear decryption status

In [26]:
# Decrypts ciphertext and displays plaintext
def decrypt_message():
    ciphertext_str = entry_ciphertext.get()
    try:
        ciphertext = [int(ciphertext_str[i:i+4]) for i in range(0, len(ciphertext_str), 4)]  # Ensure correct splitting
    except ValueError:
        messagebox.showerror("Error", "Invalid ciphertext. Please enter a valid integer sequence.")
        return
    if not private_key:
        messagebox.showerror("Error", "Please generate keys first.")
        return
    decrypted_msg = decrypt(private_key, ciphertext)
    entry_plaintext.delete(0, tk.END)
    entry_plaintext.insert(0, decrypted_msg)
    decryption_status.config(text="Decryption Successful.")
    encryption_status.config(text="")  # Clear encryption status

In [27]:
# Initialize Tkinter window
root = tk.Tk()
root.title("RSA Encryption/Decryption Tool")

''

In [28]:
def set_dark_theme():
    root.config(bg="#282828")  # Set background color
    root.tk_setPalette(background="#282828", foreground="white", activeBackground="#505050", activeForeground="white")  # Set palette colors

    # Update widget colors
    for widget in root.winfo_children():
        if isinstance(widget, tk.Label) or isinstance(widget, tk.Entry):
            widget.config(bg="#282828", fg="white")  # Set label and entry colors
        elif isinstance(widget, tk.Button):
            widget.config(bg="#505050", fg="white", activebackground="#707070", activeforeground="white")  # Set button colors
        elif isinstance(widget, tk.Toplevel):
            widget.config(bg="#282828")  # Set Toplevel window background color

In [29]:
set_dark_theme()

In [30]:
# Prime number input fields
tk.Label(root, text="Prime 1:").grid(row=0, column=0)
entry_prime1 = tk.Entry(root)
entry_prime1.grid(row=0, column=1)

tk.Label(root, text="Prime 2:").grid(row=0, column=2)
entry_prime2 = tk.Entry(root)
entry_prime2.grid(row=0, column=3)

# Generate keys button
btn_generate_keys = tk.Button(root, text="Generate Keys", command=generate_keys)
btn_generate_keys.grid(row=0, column=4)

# Display keys
public_key_var = tk.StringVar()
private_key_var = tk.StringVar()
tk.Label(root, textvariable=public_key_var).grid(row=1, columnspan=5)
tk.Label(root, textvariable=private_key_var).grid(row=2, columnspan=5)

In [31]:
# Plaintext input field
tk.Label(root, text="Plaintext:").grid(row=3, column=0)
entry_plaintext = tk.Entry(root, width=50)
entry_plaintext.grid(row=3, column=1, columnspan=3)

# Encrypt button
btn_encrypt = tk.Button(root, text="Encrypt", command=encrypt_message)
btn_encrypt.grid(row=3, column=4)

# Ciphertext input field
tk.Label(root, text="Ciphertext:").grid(row=4, column=0)
entry_ciphertext = tk.Entry(root, width=50)
entry_ciphertext.grid(row=4, column=1, columnspan=3)

# Decrypt button
btn_decrypt = tk.Button(root, text="Decrypt", command=decrypt_message)
btn_decrypt.grid(row=4, column=4)

# Encryption and decryption status labels
encryption_status = tk.Label(root, text="")
encryption_status.grid(row=5, columnspan=5)

decryption_status = tk.Label(root, text="")
decryption_status.grid(row=6, columnspan=5)

In [32]:
# Run the application
public_key = None
private_key = None
root.mainloop()